# Public Commenting in a Pandemic

## Cleaning & Analysis

In [1]:
# import packages
import pandas as pd
import numpy as np
import json
import time
import os
import datetime
import re

In [2]:
# Specify the path of the folder where the data are saved
filePath = "C:/Users/mark/Box Sync/_MF/Assignments/Insights/Public Commenting and COVID-19/Data/Annual/"

## 2020: Cleaning

In [3]:
# load CSV
fileName = 'endpoint_documents_PS_2020.csv'
with open(filePath+fileName,'r',encoding='utf-8') as loadfile:
    df2020 = pd.read_csv(loadfile, index_col='index')
df2020.info()

C:\Users\mark\anaconda3\envs\pipenv1\lib\site-packages\numpy\lib\arraysetops.py:569: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


<class 'pandas.core.frame.DataFrame'>
Int64Index: 1368244 entries, 0 to 1368243
Data columns (total 13 columns):
 #   Column                    Non-Null Count    Dtype  
---  ------                    --------------    -----  
 0   agencyAcronym             1368244 non-null  object 
 1   attachmentCount           1368244 non-null  int64  
 2   commentDueDate            0 non-null        float64
 3   commentStartDate          0 non-null        float64
 4   docketId                  1368244 non-null  object 
 5   docketType                1368244 non-null  object 
 6   documentId                1368244 non-null  object 
 7   numberOfCommentsReceived  1368244 non-null  int64  
 8   openForComment            1368244 non-null  bool   
 9   postedDate                1368244 non-null  object 
 10  submitterName             1269579 non-null  object 
 11  title                     1368244 non-null  object 
 12  organization              84303 non-null    object 
dtypes: bool(1), float64(2), int

In [4]:
# shorten/rename number of comments received column
df2020 = df2020.rename(columns={'numberOfCommentsReceived': 'commentsReceived'})

# create posted count column
df2020['commentsPosted'] = 1

df2020.loc[:,['commentsPosted','commentsReceived']].query('commentsReceived > 1')

,commentsPosted,commentsReceived
index,,
7570,1,21
8693,1,46
9029,1,86
9404,1,4868
10665,1,58
...,...,...
1183418,1,55
1184173,1,7132
1190552,1,2


In [5]:
# create list for documentId's of entries to clean
cleaning_list = []
type(cleaning_list)

list

### Dates and Months

In [6]:
# create new columns for year and month
df2020['postedYear'] = df2020['postedDate'].str.slice(start=0,stop=4)
df2020['postedMonth'] = df2020['postedDate'].str.slice(start=6,stop=7)

# convert to integers
df2020['postedYear'] = pd.to_numeric(df2020['postedYear'])
df2020['postedMonth'] = pd.to_numeric(df2020['postedMonth'])

# return new columns
print(df2020.loc[:,['postedYear','postedMonth']].dtypes)
df2020.loc[:,['postedYear','postedMonth']]

postedYear     int64
postedMonth    int64
dtype: object


,postedYear,postedMonth
index,,
0,2020,1
1,2020,1
2,2020,1
3,2020,1
4,2020,1
...,...,...
1368239,2020,5
1368240,2020,5
1368241,2020,5


In [7]:
# created new column with postedDate in datetime format
# https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.to_datetime.html
df2020['postedDatetime'] = pd.to_datetime(df2020['postedDate'], utc=True)
df2020.loc[:,['postedDate','postedDatetime']]

,postedDate,postedDatetime
index,,
0,2020-01-01T00:00:00-05:00,2020-01-01 05:00:00+00:00
1,2020-01-01T00:00:00-05:00,2020-01-01 05:00:00+00:00
2,2020-01-01T00:00:00-05:00,2020-01-01 05:00:00+00:00
3,2020-01-01T00:00:00-05:00,2020-01-01 05:00:00+00:00
4,2020-01-01T00:00:00-05:00,2020-01-01 05:00:00+00:00
...,...,...
1368239,2020-05-31T00:00:00-04:00,2020-05-31 04:00:00+00:00
1368240,2020-05-31T00:00:00-04:00,2020-05-31 04:00:00+00:00
1368241,2020-05-31T00:00:00-04:00,2020-05-31 04:00:00+00:00


In [8]:
# pivot by month
by_YearMonth = pd.pivot_table(df2020,values=['commentsPosted','commentsReceived'],
                              columns=['postedYear'],
                              index=['postedMonth'],
                              aggfunc=np.sum, margins=False)

by_YearMonth.loc[:,:]

,commentsPosted,commentsReceived
postedYear,2020,2020
postedMonth,,
1,242610,351745
2,196694,356346
3,279508,779202
4,59137,210059
5,590291,972679
6,4,4


In [9]:
# query entries with Month == 6
queries = df2020.loc[:,['postedMonth','postedDate','documentId']].query('postedMonth == 6')
queries

,postedMonth,postedDate,documentId
index,,,
363597,6,2020-06-23T00:00:00-04:00,FNS-2019-0001-31002
1036000,6,2020-06-22T00:00:00-04:00,CEQ-2019-0003-415413
1036022,6,2020-06-22T00:00:00-04:00,CEQ-2019-0003-403365
1036040,6,2020-06-22T00:00:00-04:00,CEQ-2019-0003-430544


In [10]:
# add to cleaning list
docs_to_add = {'fix_month':
             queries.loc[:,'documentId'].tolist()}
cleaning_list.append(docs_to_add)
print(cleaning_list)

[{'fix_month': ['FNS-2019-0001-31002', 'CEQ-2019-0003-415413', 'CEQ-2019-0003-403365', 'CEQ-2019-0003-430544']}]


### Agency groupings

In [11]:
by_Agency = pd.pivot_table(df2020,
                           values=['commentsPosted','commentsReceived'],
                           index=['agencyAcronym'],
                           aggfunc=np.sum, margins=False)
print(len(by_Agency))
by_Agency

106


,commentsPosted,commentsReceived
agencyAcronym,,
AID,474,474
AMS,6702,6757
APHIS,1304,1304
ATBCB,38,38
ATSDR,6,6
...,...,...
USCIS,5349,6246
USDA,37,37
USTR,28651,28653


In [12]:
agency_list = by_Agency.index.tolist()
print(len(agency_list),'\n')
print(agency_list)

106 

['AID', 'AMS', 'APHIS', 'ATBCB', 'ATSDR', 'BIA', 'BIS', 'BLM', 'BOP', 'BOR', 'BSEE', 'CCC', 'CDC', 'CEQ', 'CFPB', 'CISA', 'CMS', 'CNCS', 'COE', 'COLC', 'CPSC', 'CSB', 'DARS', 'DEA', 'DHS', 'DOC', 'DOD', 'DOE', 'DOI', 'DOJ', 'DOL', 'DOS', 'DOT', 'EAC', 'ED', 'EEOC', 'EERE', 'EOIR', 'EPA', 'ETA', 'FAA', 'FAR', 'FCIC', 'FDA', 'FEMA', 'FHWA', 'FINCEN', 'FISCAL', 'FMCSA', 'FNS', 'FRA', 'FRTIB', 'FS', 'FSA', 'FSIS', 'FTA', 'FTC', 'FWS', 'GSA', 'HHS', 'HHSIG', 'HRSA', 'HUD', 'IRS', 'ITA', 'LMSO', 'MARAD', 'MSHA', 'NARA', 'NCUA', 'NHTSA', 'NIST', 'NLRB', 'NOAA', 'NPS', 'NRC', 'NRCS', 'NTSB', 'OCC', 'OFCCP', 'OMB', 'OPM', 'OSHA', 'OSM', 'PBGC', 'PHMSA', 'PTO', 'RBS', 'RHS', 'RUS', 'SBA', 'SSA', 'TREAS', 'TSA', 'TTB', 'USA', 'USAF', 'USBC', 'USC', 'USCBP', 'USCG', 'USCIS', 'USDA', 'USTR', 'VA', 'WCPO']


In [13]:
# create dictionary for Branch to Agency lookups
branch_dict = {'Judicial': ['USC'], 
               'Legislative': ['LOC', 'COLC'], 
               'Independent': ['AID', 'ATBCB', 'CFPB', 'CNCS', 'CPSC', 'CSB', 'EAC', 
                               'EEOC', 'FRTIB', 'FTC', 'GSA', 'NARA', 'NCUA', 'NLRB', 
                               'NRC', 'NTSB', 'OPM', 'PBGC', 'SBA', 'SSA'], 
               'Executive': ['DHS', 'CISA', 'FEMA', 'TSA', 'USCBP', 'USCG', 'USCIS', 
                             'DOC', 'BIS', 'ITA', 'NIST', 'NOAA', 'PTO', 'USBC', 'DOD', 
                             'COE', 'DARS', 'USA', 'USAF', 'DOE', 'EERE', 'DOI', 'BIA', 
                             'BLM', 'BOR', 'BSEE', 'FWS', 'NPS', 'OSM', 'DOJ', 'BOP', 'DEA', 
                             'EOIR', 'DOL', 'ETA', 'LMSO', 'MSHA', 'OFCCP', 'OSHA', 'WCPO', 
                             'DOS', 'DOT', 'FAA', 'FHWA', 'FMCSA', 'FRA', 'FTA', 'MARAD', 
                             'NHTSA', 'PHMSA', 'ED', 'EOP', 'CEQ', 'OMB', 'USTR', 'EPA', 'FAR', 'HHS', 'ATSDR', 'CDC', 'CMS', 'FDA', 'HHSIG', 'HRSA', 'HUD', 'TREAS', 'FINCEN', 'FISCAL', 'IRS', 'OCC', 'TTB', 'USDA', 
                             'AMS', 'APHIS', 'CCC', 'FCIC', 'FNS', 'FS', 'FSA', 'FSIS', 
                             'NRCS', 'RBS', 'RHS', 'RUS', 'VA']
              }
print(len(branch_dict))
print(branch_dict['Independent'])
print(len(branch_dict['Judicial']+
          branch_dict['Legislative']+
          branch_dict['Independent']+
          branch_dict['Executive']) - len(['LOC','EOP']))

4
['AID', 'ATBCB', 'CFPB', 'CNCS', 'CPSC', 'CSB', 'EAC', 'EEOC', 'FRTIB', 'FTC', 'GSA', 'NARA', 'NCUA', 'NLRB', 'NRC', 'NTSB', 'OPM', 'PBGC', 'SBA', 'SSA']
106


In [14]:
%%time

# references:
    # https://stackoverflow.com/questions/49161120/pandas-python-set-value-of-one-column-based-on-value-in-another-column
    # https://stackoverflow.com/questions/30446510/list-of-elements-to-boolean-array

# create boolean arrays for each branch
bool_jud = [True if item in branch_dict['Judicial'] else False for item in df2020.loc[:,'agencyAcronym'].tolist()]
bool_leg = [True if item in branch_dict['Legislative'] else False for item in df2020.loc[:,'agencyAcronym'].tolist()]
bool_ind = [True if item in branch_dict['Independent'] else False for item in df2020.loc[:,'agencyAcronym'].tolist()]
bool_exe = [True if item in branch_dict['Executive'] else False for item in df2020.loc[:,'agencyAcronym'].tolist()]

# create new column for branch
df2020['agencyBranch'] = ''

# use boolean arrays to fill new column
df2020.loc[bool_jud,'agencyBranch'] = 'Judicial'
df2020.loc[bool_leg,'agencyBranch'] = 'Legislative'
df2020.loc[bool_ind,'agencyBranch'] = 'Independent'
df2020.loc[bool_exe,'agencyBranch'] = 'Executive'

df2020.loc[:,['agencyAcronym','agencyBranch']]

Wall time: 7.36 s


,agencyAcronym,agencyBranch
index,,
0,DOJ,Executive
1,DOJ,Executive
2,DOJ,Executive
3,DOJ,Executive
4,DOJ,Executive
...,...,...
1368239,FSA,Executive
1368240,FSA,Executive
1368241,FSA,Executive


In [15]:
# query df by branch
df2020.query('agencyBranch == ""')

,agencyAcronym,attachmentCount,commentDueDate,commentStartDate,docketId,docketType,documentId,commentsReceived,openForComment,postedDate,submitterName,title,organization,commentsPosted,postedYear,postedMonth,postedDatetime,agencyBranch
index,,,,,,,,,,,,,,,,,,


In [16]:
# query df by multiple branches
df2020.query('agencyBranch == "Legislative" | agencyBranch == "Judicial" ')

,agencyAcronym,attachmentCount,commentDueDate,commentStartDate,docketId,docketType,documentId,commentsReceived,openForComment,postedDate,submitterName,title,organization,commentsPosted,postedYear,postedMonth,postedDatetime,agencyBranch
index,,,,,,,,,,,,,,,,,,
9095,USC,1,NaN,NaN,USC-RULES-CV-2019-0003,Rulemaking,USC-RULES-CV-2019-0003-0018,1,False,2020-01-03T00:00:00-05:00,Amy Brogioli,Comment on USC-RULES-CV-2019-0003,American Association for Justice,1,2020,1,2020-01-03 05:00:00+00:00,Judicial
39766,USC,1,NaN,NaN,USC-RULES-CV-2019-0003,Rulemaking,USC-RULES-CV-2019-0003-0019,1,False,2020-01-08T00:00:00-05:00,Philip Willman,Comment on Civil Rule 7.1 by DRI,NaN,1,2020,1,2020-01-08 05:00:00+00:00,Judicial
144578,USC,1,NaN,NaN,USC-RULES-CV-2019-0003,Rulemaking,USC-RULES-CV-2019-0003-0021,1,False,2020-01-22T00:00:00-05:00,Bruce Braley,Comment on USC-RULES-CV-2019-0003,NaN,1,2020,1,2020-01-22 05:00:00+00:00,Judicial
219508,USC,1,NaN,NaN,USC-RULES-CV-2019-0003,Rulemaking,USC-RULES-CV-2019-0003-0023,1,False,2020-01-29T00:00:00-05:00,John R. Kouris,Comment on USC-RULES-CV-2019-0003,NaN,1,2020,1,2020-01-29 05:00:00+00:00,Judicial
223552,USC,1,NaN,NaN,USC-RULES-CV-2019-0003,Rulemaking,USC-RULES-CV-2019-0003-0022,1,False,2020-01-29T00:00:00-05:00,Fred Buck,Comment on USC-RULES-CV-2019-0003,American College of Trial Lawyers,1,2020,1,2020-01-29 05:00:00+00:00,Judicial
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1356190,COLC,1,NaN,NaN,COLC-2020-0005,Rulemaking,COLC-2020-0005-0011,1,False,2020-05-26T00:00:00-04:00,Danielle Aguirre,National Music Publishers' Association - Comments,National Music Publishers' Association,1,2020,5,2020-05-26 04:00:00+00:00,Legislative
1356267,COLC,1,NaN,NaN,COLC-2020-0005,Rulemaking,COLC-2020-0005-0015,1,False,2020-05-26T00:00:00-04:00,William Colitre,"Music Reports, Inc. - Comments","Music Reports, Inc.",1,2020,5,2020-05-26 04:00:00+00:00,Legislative
1356272,COLC,1,NaN,NaN,COLC-2020-0005,Rulemaking,COLC-2020-0005-0007,1,False,2020-05-26T00:00:00-04:00,Michael Lewan,Recording Academy - Comments,Recording Academy,1,2020,5,2020-05-26 04:00:00+00:00,Legislative


In [17]:
# create dict for Parent Agencies
parent_dict = dict(LOC = ['LOC', 'COLC'], 
                   DHS = ['DHS', 'CISA', 'FEMA', 'TSA', 'USCBP', 'USCG', 'USCIS'],
                   DOC = ['DOC', 'BIS', 'ITA', 'NIST', 'NOAA', 'PTO', 'USBC'],
                   DOD = ['DOD', 'COE', 'DARS', 'USA', 'USAF'],
                   DOE = ['DOE', 'EERE'],
                   DOI = ['DOI', 'BIA', 'BLM', 'BOR', 'BSEE', 'FWS', 'NPS', 'OSM'],
                   DOJ = ['DOJ', 'BOP', 'DEA', 'EOIR'],
                   DOL = ['DOL', 'ETA', 'LMSO', 'MSHA', 'OFCCP', 'OSHA', 'WCPO'],
                   DOS = ['DOS'],
                   DOT = ['DOT', 'FAA', 'FHWA', 'FMCSA', 'FRA', 'FTA', 'MARAD', 'NHTSA', 'PHMSA'],
                   ED = ['ED'],
                   EOP = ['EOP', 'CEQ', 'OMB', 'USTR'],
                   EPA = ['EPA'],
                   FAR = ['FAR'],
                   HHS = ['HHS', 'ATSDR', 'CDC', 'CMS', 'FDA', 'HHSIG', 'HRSA'],
                   HUD = ['HUD'],
                   TREAS = ['TREAS', 'FINCEN', 'FISCAL', 'IRS', 'OCC', 'TTB'],
                   USDA = ['USDA', 'AMS', 'APHIS', 'CCC', 'FCIC', 'FNS', 'FS', 'FSA', 'FSIS', 'NRCS', 'RBS', 'RHS', 'RUS'],
                   VA = ['VA']
                  )

x = 1
print(list(parent_dict.keys())[x])
print(list(parent_dict.values())[x])

DHS
['DHS', 'CISA', 'FEMA', 'TSA', 'USCBP', 'USCG', 'USCIS']


In [18]:
%%time

# create new column for parent agency
df2020['agencyParent'] = ''

# parent==acronym for judicial & independent agencies
df2020.loc[bool_jud,'agencyParent'] = df2020.loc[bool_jud,'agencyAcronym']
df2020.loc[bool_ind,'agencyParent'] = df2020.loc[bool_ind,'agencyAcronym']

# set parent for executive & legislative agencies
dictLength = len(parent_dict)
listValues = list(parent_dict.values())
listKeys = list(parent_dict.keys())

for key in range(dictLength):
    print(list(parent_dict.keys())[key])
    bool_array = [True if item in listValues[key] else False for item in df2020.loc[:,'agencyAcronym'].tolist()]
    df2020.loc[bool_array,'agencyParent'] = [listKeys[key] if item in listValues[key] else '' for item in df2020.loc[bool_array,'agencyAcronym'].tolist()]

df2020.loc[:,['agencyAcronym','agencyParent','agencyBranch']]

LOC
DHS
DOC
DOD
DOE
DOI
DOJ
DOL
DOS
DOT
ED
EOP
EPA
FAR
HHS
HUD
TREAS
USDA
VA
Wall time: 49.9 s


,agencyAcronym,agencyParent,agencyBranch
index,,,
0,DOJ,DOJ,Executive
1,DOJ,DOJ,Executive
2,DOJ,DOJ,Executive
3,DOJ,DOJ,Executive
4,DOJ,DOJ,Executive
...,...,...,...
1368239,FSA,USDA,Executive
1368240,FSA,USDA,Executive
1368241,FSA,USDA,Executive


In [19]:
df2020.loc[:,['agencyAcronym','agencyParent','agencyBranch']].query('agencyParent == ""')

,agencyAcronym,agencyParent,agencyBranch
index,,,


In [20]:
print(len(df2020.query('agencyBranch == "Independent"')) + 
      len(df2020.query('agencyBranch == "Judicial"')))

218169


In [21]:
by_AgencyParent = pd.pivot_table(df2020,
                           values=['commentsPosted','commentsReceived'],
                           index=['agencyBranch','agencyParent'],
                           aggfunc=np.sum, margins=False)
print(len(by_AgencyParent))
by_AgencyParent.query('agencyBranch == "Executive"')

40


commentsPosted  commentsReceived
agencyBranch agencyParent                                  
Executive    DHS                     6508             38136
             DOC                     8126              8126
             DOD                     8060              8060
             DOE                      183               184
             DOI                    23270             68869
             DOJ                    30260             30264
             DOL                    12172             12251
             DOS                      298               298
             DOT                    78256             78257
             ED                     18597             18597
             EOP                   750925           1175871
             EPA                    10667            758595
             FAR                       14                14
             HHS                    93167             93259
             HUD                     2397              2557
             TREAS                   3534              3534
             USDA                   98324            148829
             VA                      5251              5251

### MCC, Duplicate, or Significantly Similar Comments

In [22]:
# view MCC comments that are posted as "Representative" comments
    # e.g., "This agency received 21 duplicate or significantly similar comments."
    # Ex: https://www.regulations.gov/document?D=FNS-2019-0009-5664
lookup = ['documentId','title','organization','attachmentCount','commentsPosted','commentsReceived','agencyAcronym','agencyParent','agencyBranch']

df2020.loc[:,lookup].query('commentsReceived != commentsPosted')

,documentId,title,organization,attachmentCount,commentsPosted,commentsReceived,agencyAcronym,agencyParent,agencyBranch
index,,,,,,,,,
7570,FNS-2019-0009-5664,Comment on FR Doc # 2019-21287,Lutheran Advocacy-MN,1,1,21,FNS,USDA,Executive
8693,FNS-2019-0009-5811,unknown postcard and letter campaign,NaN,1,1,46,FNS,USDA,Executive
9029,FNS-2019-0009-5465,Comment on FR Doc # 2019-21287,Just Harvest,2,1,86,FNS,USDA,Executive
9404,FNS-2019-0009-5790,Comment on FR Doc # 2019-21287,NaN,2,1,4868,FNS,USDA,Executive
10665,FNS-2019-0009-5812,Unknown postcard campaign,NaN,1,1,58,FNS,USDA,Executive
...,...,...,...,...,...,...,...,...,...
1183418,EPA-HQ-OA-2018-0259-11820,Mass Comment Campaign sponsoring organization ...,NaN,1,1,55,EPA,EPA,Executive
1184173,EPA-HQ-OA-2018-0259-11827,Mass Comment Campaign Sponsored by American Lu...,NaN,1,1,7132,EPA,EPA,Executive
1190552,SBA-2020-0022-0188,Comment on FR Doc # 2020-09398,NaN,0,1,2,SBA,SBA,Independent


In [23]:
# create bool array for agency-marked MCCs (i.e., representative comments)
bool_MCC = df2020['commentsPosted']!=df2020['commentsReceived']
print(bool_MCC.value_counts(),'\n')

# create bool array for comments to group as representative
    # reference for regex: https://docs.python.org/3/howto/regex.html
bool_group = df2020.loc[:,'title'].str.contains('MM[\d]+|Mass Mail|Mass Comment', regex=True, case=True)
print(bool_group.value_counts(),'\n')

# create bool array for comments that overlap (both R&G)
bool_RnG = bool_MCC & bool_group
print(bool_RnG.value_counts(),'\n')

# create new column
df2020['MCCfilter'] = 'Unique'

# use boolean arrays to fill new column
df2020.loc[bool_MCC,'MCCfilter'] = 'Representative'
df2020.loc[bool_group,'MCCfilter'] = 'Grouped'
df2020.loc[bool_RnG,'MCCfilter'] = 'Both R&G'

# convert new column to categorical
    # https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.astype.html#pandas.DataFrame.astype
df2020 = df2020.astype({'MCCfilter': 'category'})
print(df2020.loc[:,'MCCfilter'].value_counts())

df2020.loc[:,['documentId','docketId','agencyAcronym','title','commentsPosted','commentsReceived','MCCfilter']]

False    1368037
True         207
dtype: int64 

True     712193
False    656051
Name: title, dtype: int64 

False    1368151
True          93
dtype: int64 

Grouped           712100
Unique            655937
Representative       114
Both R&G              93
Name: MCCfilter, dtype: int64


,documentId,docketId,agencyAcronym,title,commentsPosted,commentsReceived,MCCfilter
index,,,,,,,
0,DOJ-OAG-2019-0004-18252,DOJ-OAG-2019-0004,DOJ,Comment on FR Doc # 2019-22877,1,1,Unique
1,DOJ-OAG-2019-0004-18249,DOJ-OAG-2019-0004,DOJ,Comment on FR Doc # 2019-22877,1,1,Unique
2,DOJ-OAG-2019-0004-20423,DOJ-OAG-2019-0004,DOJ,Comment on FR Doc # 2019-22877,1,1,Unique
3,DOJ-OAG-2019-0004-18246,DOJ-OAG-2019-0004,DOJ,Comment on FR Doc # 2019-22877,1,1,Unique
4,DOJ-OAG-2019-0004-20425,DOJ-OAG-2019-0004,DOJ,Comment on FR Doc # 2019-22877,1,1,Unique
...,...,...,...,...,...,...,...
1368239,FSA-2020-0004-0272,FSA-2020-0004,FSA,Comment on FR Doc # 2020-11155,1,1,Unique
1368240,FSA-2020-0004-0282,FSA-2020-0004,FSA,Comment on FR Doc # 2020-11155,1,1,Unique
1368241,FSA-2020-0004-0284,FSA-2020-0004,FSA,Comment on FR Doc # 2020-11155,1,1,Unique


In [24]:
df2020.loc[:,['MCCfilter','title','commentsPosted','commentsReceived']].query('MCCfilter == "Both R&G"')

,MCCfilter,title,commentsPosted,commentsReceived
index,,,,
22689,Both R&G,Mass Comment Campaign sponsored by Pipeliners ...,1,3285
24737,Both R&G,Mass Comment Campaign sponsored by Absolute En...,1,7
28256,Both R&G,Mass Comment Campaign sponsored by Environment...,1,2779
133197,Both R&G,Mass Mail Campaign 4: Comment Submitted by Cit...,1,12
152604,Both R&G,Mass Comment Campaign submitted by Natural Res...,1,20643
...,...,...,...,...
1180981,Both R&G,Mass Comment Campaign sponsored by Moms Clean ...,1,4188
1183418,Both R&G,Mass Comment Campaign sponsoring organization ...,1,55
1184173,Both R&G,Mass Comment Campaign Sponsored by American Lu...,1,7132


In [25]:
by_MCCfilter = pd.pivot_table(df2020, values=['commentsPosted','commentsReceived'],
                              index=['MCCfilter'],
                              aggfunc=np.sum, margins=True)

by_MCCfilter

,commentsPosted,commentsReceived
MCCfilter,,
Both R&G,93,859119
Grouped,712100,712100
Representative,114,442879
Unique,655937,655937
All,1368244,2670035


In [26]:
# filter comments for grouping
MMfilter = df2020.loc[np.array(df2020['MCCfilter']=='Grouped') | np.array(df2020['MCCfilter']=='Both R&G'),:]

pd.pivot_table(MMfilter, 
               values=['commentsPosted','commentsReceived'], 
               index=['docketId'], 
               aggfunc=np.sum)

,commentsPosted,commentsReceived
docketId,,
CEQ-2019-0003,711850,815252
DHS-2019-0049,4,4
EPA-HQ-OA-2018-0259,38,325243
EPA-HQ-OAR-2004-0394,1,217
EPA-HQ-OAR-2010-0505,2,8481
EPA-HQ-OAR-2017-0355,2,81
EPA-HQ-OAR-2017-0757,5,448
EPA-HQ-OAR-2018-0283,1,145
EPA-HQ-OAR-2018-0746,2,5422


In [27]:
# determine number of MCC campaigns in docket and verify tally of comments
docket = 'CEQ-2019-0003'
restriction = MMfilter['docketId']==docket
obs = 1
num = 0
massTally = num
obsTally = []

while obs > 0:
    # use two different regex patterns -- Mass Mail Campaign #\\b and MMx#\\b
    bool_lookup = MMfilter.loc[restriction,'title'].str.contains('Mass Mail Campaign '+str(num)+r'\b|MM'+str(num)+r'\b',
                                                                 regex=True, case=True)
    
    try: # try 1
        obs = int(bool_lookup.value_counts()[True])
        print('MCC '+str(num)+' -- obs = '+str(obs))
        massTally = num
        obsTally.extend([obs])
    except:
        print('Error occurred for MCC'+str(num))
        try: # try 2
            num = num + 1
            bool_lookup = MMfilter.loc[restriction,'title'].str.contains('Mass Mail Campaign '+str(num)+r'\b|MM'+str(num)+r'\b', regex=True, case=True)
            obs = int(bool_lookup.value_counts()[True])
        except:
            print('Error occurred for MCC'+str(num))
            try: # try 3
                num = num + 1
                bool_lookup = MMfilter.loc[MMfilter['docketId']!=docket,'title'].str.contains('Mass Mail Campaign '+str(num)+r'\b|MM'+str(num)+r'\b', regex=True, case=True)
                obs = int(bool_lookup.value_counts()[True])
            except:
                print('Error occurred for MCC'+str(num))
                obs = 0
    else:
        num = num + 1

print('MCC tally = '+str(massTally)+' -- obs tally = '+str(sum(obsTally)))
print(len(MMfilter.loc[restriction,:]))

Error occurred for MCC0
MCC 1 -- obs = 25481
MCC 2 -- obs = 19073
MCC 3 -- obs = 14985
MCC 4 -- obs = 15270
MCC 5 -- obs = 17137
MCC 6 -- obs = 8632
MCC 7 -- obs = 11468
MCC 8 -- obs = 7233
MCC 9 -- obs = 5710
MCC 10 -- obs = 5506
MCC 11 -- obs = 4463
MCC 12 -- obs = 4373
MCC 13 -- obs = 4795
MCC 14 -- obs = 3656
MCC 15 -- obs = 32
MCC 16 -- obs = 4188
MCC 17 -- obs = 2582
MCC 18 -- obs = 1354
MCC 19 -- obs = 1069
MCC 20 -- obs = 908
MCC 21 -- obs = 726
MCC 22 -- obs = 693
MCC 23 -- obs = 663
MCC 24 -- obs = 603
MCC 25 -- obs = 537
MCC 26 -- obs = 651
MCC 27 -- obs = 715
MCC 28 -- obs = 413
MCC 29 -- obs = 388
MCC 30 -- obs = 355
MCC 31 -- obs = 326
MCC 32 -- obs = 278
MCC 33 -- obs = 267
MCC 34 -- obs = 256
MCC 35 -- obs = 278
MCC 36 -- obs = 233
MCC 37 -- obs = 246
MCC 38 -- obs = 178
MCC 39 -- obs = 203
MCC 40 -- obs = 152
MCC 41 -- obs = 148
MCC 42 -- obs = 143
MCC 43 -- obs = 112
MCC 44 -- obs = 112
MCC 45 -- obs = 797
MCC 46 -- obs = 203
MCC 47 -- obs = 105
MCC 48 -- obs = 177
MC

In [28]:
%%time

# mission: create new DataFrame of CEQ MCC campaigns for merging with df2020

# create lists for populating df columns
idList = [] ## documentId
titleList = [] ## title
numList = [] ## MCC number
obsList = [] ## comments posted for an MCC

# populate lists with for loop and regex
for num in range(massTally+1):
    regex_search = 'Mass Mail Campaign '+str(num)+r'\b|MM'+str(num)+r'\b'
    bool_search = MMfilter.loc[restriction,'title'].str.contains(regex_search,
                                                                 regex=True, case=True)
    try: # try 1
        idList.extend(MMfilter.loc[restriction,:].loc[bool_search,'documentId'].tolist())
        titleList.extend(MMfilter.loc[restriction,:].loc[bool_search,'title'].tolist())
        obs = int(bool_search.value_counts()[True])
        obsList.extend([obs]*obs)
        numList.extend([num]*obs)
        if num%50==0:
            print('Just finished MCC'+str(num))
    except:
        print('Error occurred for MCC'+str(num))
        continue

# zip lists into new list and generate df
dataList = list(zip(idList,titleList,numList,obsList))
dfCEQMCC = pd.DataFrame(dataList, columns = ['documentId', 'title', 'MCCnumber', 'commentsGrouped'])

# check whether length of new df is correct
if len(dfCEQMCC) == len(MMfilter.loc[restriction,:]):
    print(dfCEQMCC.info())
else:
    print('Check DataFrame before merge. It might be missing entries.')

Error occurred for MCC0
Just finished MCC50
Just finished MCC100
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 711850 entries, 0 to 711849
Data columns (total 4 columns):
 #   Column           Non-Null Count   Dtype 
---  ------           --------------   ----- 
 0   documentId       711850 non-null  object
 1   title            711850 non-null  object
 2   MCCnumber        711850 non-null  int64 
 3   commentsGrouped  711850 non-null  int64 
dtypes: int64(2), object(2)
memory usage: 21.7+ MB
None
Wall time: 4min 1s


In [29]:
print(len(df2020))
df2020Grouped = df2020.merge(dfCEQMCC, how='outer', on=['documentId','title'], indicator=True, validate='1:1')
df2020Grouped.info()

1368244
<class 'pandas.core.frame.DataFrame'>
Int64Index: 1368244 entries, 0 to 1368243
Data columns (total 23 columns):
 #   Column            Non-Null Count    Dtype              
---  ------            --------------    -----              
 0   agencyAcronym     1368244 non-null  object             
 1   attachmentCount   1368244 non-null  int64              
 2   commentDueDate    0 non-null        float64            
 3   commentStartDate  0 non-null        float64            
 4   docketId          1368244 non-null  object             
 5   docketType        1368244 non-null  object             
 6   documentId        1368244 non-null  object             
 7   commentsReceived  1368244 non-null  int64              
 8   openForComment    1368244 non-null  bool               
 9   postedDate        1368244 non-null  object             
 10  submitterName     1269579 non-null  object             
 11  title             1368244 non-null  object             
 12  organization      84

In [30]:
pd.pivot_table(df2020Grouped, values=['commentsGrouped','commentsPosted'],
               index=['postedMonth'], columns=['MCCfilter'],
               aggfunc='count', dropna=False, fill_value=0, margins=True)

commentsGrouped                                        \
MCCfilter          Both R&G Grouped Representative Unique     All   
postedMonth                                                         
1                         0    3010              0      0    3010   
2                         0   12229              0      0   12229   
3                         3  149479              0      0  149482   
4                         0       0              0      0       0   
5                         3  547123              0      0  547126   
6                         0       3              0      0       3   
All                       6  711844              0      0  711850   

            commentsPosted                                          
MCCfilter         Both R&G Grouped Representative  Unique      All  
postedMonth                                                         
1                       13    3148             39  239410   242610  
2                       17   12265              7  184405   196694  
3                       19  149499             42  129948   279508  
4                       13      47             10   59067    59137  
5                       31  547138             16   43106   590291  
6                        0       3              0       1        4  
All                     93  712100            114  655937  1368244

In [31]:
pd.pivot_table(df2020Grouped, values=['commentsGrouped','commentsPosted'],
               index=['_merge'],
               aggfunc='count', dropna=False, fill_value=0, margins=True)

,commentsGrouped,commentsPosted
_merge,,
left_only,0,656394
right_only,0,0
both,711850,711850
All,711850,1368244


#### Assign representative comments for MCCs

In [32]:
# create new column for representative comment
df2020Grouped['represent'] = 'Fill'

bool_nan = df2020Grouped['_merge']=='left_only'
df2020Grouped.loc[bool_nan,'represent'] = 'Unidentified' ## for non-MCCs or those that haven't been identified

bool_rep = df2020Grouped.loc[(~bool_nan),'title'].str.contains('Mass Mail Campaign '+r'\b', regex=True, case=True)
df2020Grouped.loc[bool_rep&(~bool_nan),['represent']] = 'Yes' ## for representative comments

bool_notrep = df2020Grouped.loc[(~bool_nan),'title'].str.contains('MM[\d]+ Comment'+r'\b', regex=True, case=True)
df2020Grouped.loc[bool_notrep&(~bool_nan),['represent']] = 'No' ## for grouped comments that aren't representative

print(bool_nan.value_counts(), 
      bool_rep.value_counts(), 
      bool_notrep.value_counts(), sep='\n\n')

df2020Grouped = df2020Grouped.astype({'represent': 'category'})
print('\n', 
      df2020Grouped.loc[:,'represent'].value_counts())

# pivot against MCC filter
pd.pivot_table(df2020Grouped, index=['represent'], values=['documentId'], columns=['MCCfilter'],
               aggfunc='count', fill_value=0, margins=True)

False    711850
True     656394
Name: _merge, dtype: int64

False    711731
True        119
Name: title, dtype: int64

True     711731
False       119
Name: title, dtype: int64

 No              711731
Unidentified    656394
Yes                119
Name: represent, dtype: int64


documentId                                        
MCCfilter      Both R&G Grouped Representative  Unique      All
represent                                                      
No                    4  711727              0       0   711731
Unidentified         87     256            114  655937   656394
Yes                   2     117              0       0      119
All                  93  712100            114  655937  1368244

In [61]:
df2020Grouped.query('represent == "Yes"')

,agencyAcronym,attachmentCount,commentDueDate,commentStartDate,docketId,docketType,documentId,commentsReceived,openForComment,postedDate,...,postedYear,postedMonth,postedDatetime,agencyBranch,agencyParent,MCCfilter,MCCnumber,commentsGrouped,_merge,represent
141062,CEQ,0,NaN,NaN,CEQ-2019-0003,Rulemaking,CEQ-2019-0003-0069,1,False,2020-01-21T00:00:00-05:00,...,2020,1,2020-01-21 05:00:00+00:00,Executive,EOP,Grouped,80.0,11.0,both,Yes
170762,CEQ,0,NaN,NaN,CEQ-2019-0003,Rulemaking,CEQ-2019-0003-0746,1,False,2020-01-23T00:00:00-05:00,...,2020,1,2020-01-23 05:00:00+00:00,Executive,EOP,Grouped,14.0,3656.0,both,Yes
180190,CEQ,0,NaN,NaN,CEQ-2019-0003,Rulemaking,CEQ-2019-0003-3615,1,False,2020-01-24T00:00:00-05:00,...,2020,1,2020-01-24 05:00:00+00:00,Executive,EOP,Grouped,81.0,11.0,both,Yes
181291,CEQ,0,NaN,NaN,CEQ-2019-0003,Rulemaking,CEQ-2019-0003-3488,1,False,2020-01-24T00:00:00-05:00,...,2020,1,2020-01-24 05:00:00+00:00,Executive,EOP,Grouped,114.0,4.0,both,Yes
333493,CEQ,1,NaN,NaN,CEQ-2019-0003,Rulemaking,CEQ-2019-0003-8349,1,False,2020-02-19T00:00:00-05:00,...,2020,2,2020-02-19 05:00:00+00:00,Executive,EOP,Grouped,83.0,9.0,both,Yes
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1231423,CEQ,0,NaN,NaN,CEQ-2019-0003,Rulemaking,CEQ-2019-0003-580314,1,False,2020-05-23T00:00:00-04:00,...,2020,5,2020-05-23 04:00:00+00:00,Executive,EOP,Grouped,74.0,5412.0,both,Yes
1254250,CEQ,0,NaN,NaN,CEQ-2019-0003,Rulemaking,CEQ-2019-0003-597200,1,False,2020-05-23T00:00:00-04:00,...,2020,5,2020-05-23 04:00:00+00:00,Executive,EOP,Grouped,79.0,33.0,both,Yes
1258433,CEQ,0,NaN,NaN,CEQ-2019-0003,Rulemaking,CEQ-2019-0003-574142,1,False,2020-05-23T00:00:00-04:00,...,2020,5,2020-05-23 04:00:00+00:00,Executive,EOP,Grouped,88.0,186.0,both,Yes
1261139,CEQ,0,NaN,NaN,CEQ-2019-0003,Rulemaking,CEQ-2019-0003-580311,1,False,2020-05-23T00:00:00-04:00,...,2020,5,2020-05-23 04:00:00+00:00,Executive,EOP,Grouped,90.0,37.0,both,Yes


In [34]:
# ----- Create new DataFrame of Representative Comments ----- #

# create list of MMC, docId of earliest postedDate in MCC, docId of latest postedDate in MCC
MCCList = []
postedFirst = []
firstDate = []
postedLast = []
lastDate = []

for MCC in range(1,120):
    MCCList.extend([MCC])
    postedFirst.extend(df2020Grouped.query('MCCnumber == @MCC').
                       sort_values('postedDate', ascending=True, na_position='last').head(1)['documentId'])
    firstDate.extend(df2020Grouped.query('MCCnumber == @MCC').
                     sort_values('postedDate', ascending=True, na_position='last').head(1)['postedDate'])
    postedLast.extend(df2020Grouped.query('MCCnumber == @MCC').
                      sort_values('postedDate', ascending=False).head(1)['documentId'])
    lastDate.extend(df2020Grouped.query('MCCnumber == @MCC').
                    sort_values('postedDate', ascending=False, na_position='last').head(1)['postedDate'])


# ----- Retrieve receivedDates for representative comments ----- #
import requests

# general variables for setting parameters
APIkey = "fYTx9mVjuwc2ZSsdqmbgdtSqx7HGUd3aCRkiH6bC"
baseURL = "https://api.data.gov:443/regulations/v3/document.json?"
dctId = ""

# set parameters
params = {'api_key': APIkey,
          'documentId': dctId}

# 1) first using postedFirst list
range_first = len(postedFirst)
receivedFirst = [] # list for adding receivedDate of each postedFirst entry

# retrieve comments using Requests library and check GET request response 
for d in range(range_end):
    dctId = postedFirst[d]
    params.update({'documentId': dctId})

    dct_response = requests.get(baseURL, params=params)
    RL_remaining = int(dct_response.headers['X-RateLimit-Remaining'])

    if dct_response.status_code != 200:
        print('code '+str(dct_response.status_code)+' for page #'+str(pageIndex), 
              dct_response.text, sep='\n')
    if RL_remaining < 10:
        print('Rate Limit remaining: '+str(RL_remaining),
              "sleeping 1 minute...", sep='\n')
        time.sleep(60)

    this_receivedDate = dct_response.json()['receivedDate']
    receivedFirst.append(this_receivedDate)

print('Length of receivedFirst is '+str(len(receivedFirst)))

# 2) second using postedLast
range_last = len(postedLast)
receivedLast = [] # list for adding receivedDate of each postedLast entry

# retrieve comments using Requests library and check GET request response 
for d in range(range_last):
    dctId = postedLast[d]
    params.update({'documentId': dctId})

    dct_response = requests.get(baseURL, params=params)
    RL_remaining = int(dct_response.headers['X-RateLimit-Remaining'])

    if dct_response.status_code != 200:
        print('code '+str(dct_response.status_code)+' for page #'+str(pageIndex), 
              dct_response.text, sep='\n')
    if RL_remaining < 10:
        print('Rate Limit remaining: '+str(RL_remaining),
              "sleeping 1 minute...", sep='\n')
        time.sleep(60)

    this_receivedDate = dct_response.json()['receivedDate']
    receivedLast.append(this_receivedDate)

print('Length of receivedLast is '+str(len(receivedLast)))

# ----- Generate df from the lists ----- #
dateList = list(zip(MCCList, postedFirst, firstDate, receivedFirst, postedLast, lastDate, receivedLast))
dfpostedDate = pd.DataFrame(dateList, columns = ['MCCnumber', 'postedFirst', '', 'firstDate', 'postedLast', 'lastDate', ''])
dfpostedDate['represent'] = 'Yes'
dfpostedDate = dfpostedDate.astype({'represent': 'category'})
dfpostedDate.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 119 entries, 0 to 118
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype   
---  ------       --------------  -----   
 0   MCCnumber    119 non-null    int64   
 1   postedFirst  119 non-null    object  
 2   firstDate    119 non-null    object  
 3   postedLast   119 non-null    object  
 4   lastDate     119 non-null    object  
 5   represent    119 non-null    category
dtypes: category(1), int64(1), object(4)
memory usage: 5.0+ KB


## 2020: Analysis

In [ ]:
pd.pivot_table(df2020Grouped, values=['commentsPosted','commentsReceived'],
                              index=['MCCfilter'],
                              aggfunc=np.count, margins=True)


###### break

In [ ]:
# by_YearMonth.plot(y='documentCount', kind='bar')